In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [14]:
# Used to reset lists 
price= []
mkmodel = []
mileage= []
year = []

In [15]:
url = "https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]=&maximum_distance=20&models[]=&stock_type=used&zip=53151"

#Headers needed for requests to spoof typical PC usage (requests gives User-Agent = python by default, many sites block this access). 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
           "Accept-Encoding":"gzip, deflate",  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [16]:
def pagelooper(zipcode):
    # Uses BS4 to find total results available for zipcode given, then floor divides by 100 to find amount of 100 entry pages
    getcount = requests.get(f"""https://www.cars.com/shopping/results/?page=1&page_size=100&list_price_max=&makes[]
    =&maximum_distance=20&models[]=&stock_type=used&zip={zipcode}""",headers=headers)
    setup1 = BeautifulSoup(getcount.content,"html.parser")
    setup2 = BeautifulSoup(setup1.prettify(),'html.parser')
    pagecounter = setup2.find(class_="total-filter-count").get_text()
    x =  ''.join(re.findall(r"\d+", pagecounter.strip()))
    x2 = int(x) // 100 -1
    print(f"{x2-1} pages to print...")
    
    # Uses cars.com link scheme to determine range to loop through
    for n in range(1,x2):
        while(1):
            try:
                page = requests.get(f"""https://www.cars.com/shopping/results/?page={n}&page_size=100&list_price_max=&makes[]
                =&maximum_distance=20&models[]=&stock_type=used&zip={zip}""",headers=headers)
                
                # BS4 to find_all pertinent information from url html
                soup1 = BeautifulSoup(page.content,"html.parser")
                soup2= BeautifulSoup(soup1.prettify(),'html.parser')
                search = soup2.find_all(class_="primary-price")
                search2 = soup2.find_all(class_="title")
                search3 = soup2.find_all(class_='mileage')
            
            # AttributeError exception to allow occasional site request error
            except AttributeError:
                print('Retrying')
                time.sleep(3)
            else:
                break
        
        # Loops through prior html searches to apply get_text method to each individual entry on cars.com 
        for i,j,k in zip(search,search2,search3,):
            price.append(i.get_text().strip())
            year.append(j.get_text().strip()[0:4])
            mkmodel.append(j.get_text().strip()[5:])
            mileage.append(k.get_text().strip())
        # Sleep timer to avoid overloading site/ automatic excessive request block
        time.sleep(2)
        print(f"Page {n} scanned...")
    print("Done!")

In [17]:
pagelooper(53151)

84 pages to print...
Page 1 scanned...
Page 2 scanned...
Page 3 scanned...
Page 4 scanned...
Page 5 scanned...
Page 6 scanned...
Page 7 scanned...
Page 8 scanned...
Page 9 scanned...
Page 10 scanned...
Page 11 scanned...
Page 12 scanned...
Page 13 scanned...
Page 14 scanned...
Page 15 scanned...
Page 16 scanned...
Page 17 scanned...
Page 18 scanned...
Page 19 scanned...
Page 20 scanned...
Page 21 scanned...
Page 22 scanned...
Page 23 scanned...
Page 24 scanned...
Page 25 scanned...
Page 26 scanned...
Page 27 scanned...
Page 28 scanned...
Page 29 scanned...
Page 30 scanned...
Page 31 scanned...
Page 32 scanned...
Page 33 scanned...
Page 34 scanned...
Page 35 scanned...
Page 36 scanned...
Page 37 scanned...
Page 38 scanned...
Page 39 scanned...
Page 40 scanned...
Page 41 scanned...
Page 42 scanned...
Page 43 scanned...
Page 44 scanned...
Page 45 scanned...
Page 46 scanned...
Page 47 scanned...
Page 48 scanned...
Page 49 scanned...
Page 50 scanned...
Page 51 scanned...
Page 52 scanned...


In [18]:
#cardatas creates a list of the lists generated by pagelooper function
cardatas= list(zip(price,year,mkmodel,mileage))

# pandas dataframe implemented to simplify data analysis
need = pd.DataFrame(cardatas,columns=['Price','Year','Make/Model','Mileage'])

#all regex and numpy expressions to format entries for easier parsing
need['Mileage'] = need['Mileage'].str.replace(r'\D+','').astype(int)
need.loc[need['Price'] == 'Not Priced', 'Price'] = '0'
need['Price'] = need['Price'].str.replace(r"[$,]",'').astype(int)
need['Price'] = need['Price'].replace(0,np.NaN)
need['Price'] = need['Price'].astype('Int64')
need['Year'] = need['Year'].astype(int)
need

<ipython-input-18-5e3a55bae9e9>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  need['Mileage'] = need['Mileage'].str.replace(r'\D+','').astype(int)
<ipython-input-18-5e3a55bae9e9>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  need['Price'] = need['Price'].str.replace(r"[$,]",'').astype(int)


,Price,Year,Make/Model,Mileage
0,53500,2019,Tesla Model 3 Long Range,16210
1,29000,2018,Honda Accord Touring,12406
2,45000,2018,Ford F-150 Lariat,46763
3,34609,2018,Ford Explorer Limited,28119
4,24791,2020,Toyota Camry SE,14453
...,...,...,...,...
8289,34491,2019,Acura RDX Technology Package,17470
8290,32780,2019,Honda CR-V EX-L,12815
8291,8998,2007,Acura RL,98819
8292,33598,2018,Lincoln MKX Reserve,65635


In [19]:
need.to_csv('carscom.csv', index=False)

In [20]:
need.describe()

,Price,Year,Mileage
count,8294.000000,8294.000000,8294.000000
mean,36891.679166,2017.379310,41134.352303
std,21027.090501,3.058627,32086.786021
min,1000.000000,1984.000000,9.000000
25%,25404.500000,2017.000000,18518.750000
50%,33998.000000,2018.000000,32550.500000
75%,44527.500000,2019.000000,51216.500000
max,589900.000000,2022.000000,300000.000000
